# Imports

The reference is based on the following Keras implementation

https://keras.io/examples/nlp/text_classification_with_transformer/

In [ ]:
pip install -q datasets

In [ ]:
pip install -q transformers

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
from datasets import load_dataset
sst2 = load_dataset('glue', 'sst2')

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
#Obtain only 10% of the sets
train_small = sst2["train"].shuffle(seed=1002).select([i for i in list(range(int(len(sst2["train"])*0.2)))])
valid_small = sst2["validation"].select([i for i in list(range(int(len(sst2["validation"])*0.2)))])
test_small = sst2["validation"].select([i for i in list(range(int(len(sst2["validation"])*0.2),int(len(sst2["validation"])*0.4)))])

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-98c11ee4a0e89078.arrow


In [ ]:
checkpoint = "bert-base-uncased"
from transformers import AutoTokenizer, DataCollatorWithPadding
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize_function(x):
    return tokenizer(x["sentence"], truncation=True)#return_tensors = 'pt'

In [ ]:
tokenized_train = train_small.map(tokenize_function, batched=True)
tokenized_valid = valid_small.map(tokenize_function, batched=True)
tokenized_test = test_small.map(tokenize_function, batched=True)

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_train

Dataset({
    features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 13469
})

In [ ]:
type(tokenized_train["sentence"])

list

In [ ]:
tokenizer = Tokenizer(num_words = 20000)
def keras_tokenizer(x):
  tokenizer.fit_on_texts(x["sentence"])
  return tokenizer.texts_to_sequences(x["sentence"])

In [ ]:
train_x_hf = keras_tokenizer(tokenized_train)
valid_x_hf = keras_tokenizer(tokenized_valid)
test_x_hf = keras_tokenizer(tokenized_test)

In [ ]:
#Use input_ids as the pre-processed
#tokenized_train["input_ids"][:5]
'''
import numpy as np
train_x_hf = np.array(tokenized_train["input_ids"])
valid_x_hf = np.array(tokenized_valid["input_ids"])
test_x_hf = np.array(tokenized_test["input_ids"])
'''

'\nimport numpy as np\ntrain_x_hf = np.array(tokenized_train["input_ids"])\nvalid_x_hf = np.array(tokenized_valid["input_ids"])\ntest_x_hf = np.array(tokenized_test["input_ids"])\n'

In [ ]:
train_y_hf = np.array(tokenized_train["label"])
valid_y_hf = np.array(tokenized_valid["label"])
test_y_hf = np.array(tokenized_test["label"])

In [ ]:
test_y_hf

array([1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0])